<a href="https://colab.research.google.com/github/ParinazBinandeh1986/Environmental_Sound_Classification/blob/main/MAFnet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧠 MAFnet Full Implementation Pipeline for AVE Dataset
# Based on the paper:
# "Multi-Level Attention Fusion Network for Audio-Visual Event Recognition"
# --------------------------------------------------------------
# This Colab notebook guides you through the complete process of reproducing
# the MAFnet model for audio-visual event recognition, step by step.
# It includes dataset setup, feature extraction, model implementation,
# training, and final visualization.
# --------------------------------------------------------------

In [ ]:
# ============================================================
# STEP 1️⃣ Mount Google Drive and Define Paths
# ============================================================
# Mount your Google Drive to access the AVE dataset and save model outputs.
from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_PATH = "/content/drive/MyDrive/MAFnet"
DATASET_PATH = "/content/drive/MyDrive/AVE"
FEATURE_PATH = f"{DATASET_PATH}/features"
LABELS_CSV   = f"{DATASET_PATH}/labels.csv"

os.makedirs(PROJECT_PATH, exist_ok=True)
os.makedirs(FEATURE_PATH, exist_ok=True)
print("✅ Project folders ready. You can now proceed with feature extraction and model training.")

In [ ]:
# ============================================================
# STEP 2️⃣ Install Dependencies
# ============================================================
# Install all required Python libraries for the MAFnet pipeline.
!pip install tensorflow==2.17.0 librosa vggish opencv-python tqdm matplotlib numpy pandas scikit-learn

In [ ]:
# ============================================================
# STEP 3️⃣ Import Libraries
# ============================================================
# Load all necessary libraries for data processing, modeling, and visualization.
import numpy as np
import pandas as pd
import librosa, cv2, os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tqdm import tqdm
from vggish import VGGish
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
# ============================================================
# STEP 4️⃣ Feature Extraction (Visual + Audio)
# ============================================================
# Extract features using DenseNet (for visual) and VGGish (for audio) as described in Section 4.2 of the paper.

# ---- Visual extractor ----
vis_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg')
vis_model.trainable = False

def extract_visual_features(video_path, T=10, frames_per_clip=16):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
    clip_len = total_frames // T
    features = []
    for t in range(T):
        start = t * clip_len
        frames = []
        for i in range(frames_per_clip):
            frame_id = int(start + i * clip_len / frames_per_clip)
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
            ret, frame = cap.read()
            if not ret: continue
            frame = cv2.resize(frame, (224, 224))
            frame = preprocess_input(frame.astype(np.float32))
            frames.append(frame)
        if len(frames) > 0:
            frames = np.array(frames)
            clip_feat = vis_model.predict(frames, verbose=0)
            clip_feat = np.mean(clip_feat, axis=0)
            features.append(clip_feat)
    cap.release()
    return np.array(features)

# ---- Audio extractor ----
vggish_model = VGGish()
vggish_model.trainable = False

def extract_audio_features(video_path, T=10):
    tmp_audio = "/tmp/temp_audio.wav"
    os.system(f"ffmpeg -loglevel quiet -y -i '{video_path}' -vn -acodec pcm_s16le -ar 16000 -ac 1 {tmp_audio}")
    y, sr = librosa.load(tmp_audio, sr=16000)
    total_len = len(y)
    clip_len = total_len // T
    feats = []
    for t in range(T):
        seg = y[t*clip_len:(t+1)*clip_len]
        if len(seg) < 1000: continue
        mel = librosa.feature.melspectrogram(y=seg, sr=sr, n_mels=96)
        mel = librosa.power_to_db(mel)
        mel = np.expand_dims(mel, axis=(0, -1))
        audio_feat = vggish_model.predict(mel, verbose=0)
        feats.append(audio_feat[0])
    return np.array(feats)

# ---- Process dataset ----
def process_AVE_dataset(video_dir, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    for video_name in tqdm(os.listdir(video_dir)):
        if not video_name.endswith('.mp4'):
            continue
        base = os.path.splitext(video_name)[0]
        vpath = os.path.join(video_dir, video_name)
        try:
            vfeat = extract_visual_features(vpath)
            afeat = extract_audio_features(vpath)
            np.save(os.path.join(save_dir, f"{base}_visual.npy"), vfeat)
            np.save(os.path.join(save_dir, f"{base}_audio.npy"),  afeat)
        except Exception as e:
            print("Error processing", video_name, ":", e)

In [ ]:
# ============================================================
# STEP 5️⃣ Build the MAFnet Architecture
# ============================================================
# Build the core MAFnet structure including FiLM, Temporal Attention, and Modality Attention.

class FiLM(layers.Layer):
    def __init__(self, feature_dim):
        super().__init__()
        self.gamma_dense = layers.Dense(feature_dim)
        self.beta_dense  = layers.Dense(feature_dim)
    def call(self, inputs):
        audio_feat, visual_feat = inputs
        gamma = self.gamma_dense(visual_feat)
        beta  = self.beta_dense(visual_feat)
        return gamma * audio_feat + beta

class TemporalAttention(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.fc = layers.Dense(units, activation='relu')
        self.score = layers.Dense(1)
    def call(self, features):
        x = self.fc(features)
        s = tf.nn.softmax(self.score(x), axis=1)
        out = tf.reduce_sum(s * features, axis=1)
        return out, s

class ModalityAttention(layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.fc = layers.Dense(units, activation='relu')
        self.score = layers.Dense(1)
    def call(self, modality_feats):
        stacked = tf.stack(modality_feats, axis=1)
        x = self.fc(stacked)
        s = tf.nn.softmax(self.score(x), axis=1)
        out = tf.reduce_sum(s * stacked, axis=1)
        return out, s

def build_mafnet(T=10, feature_dim=512, num_classes=28):
    visual_inp = layers.Input(shape=(T, 1920))
    audio_inp  = layers.Input(shape=(T, 128))
    v_proj = layers.TimeDistributed(layers.Dense(feature_dim, activation='relu'))(visual_inp)
    a_proj = layers.TimeDistributed(layers.Dense(feature_dim, activation='relu'))(audio_inp)
    film = FiLM(feature_dim)
    a_film = layers.TimeDistributed(film)([a_proj, v_proj])
    t_v = TemporalAttention(feature_dim)
    t_a = TemporalAttention(feature_dim)
    v_sum, _ = t_v(v_proj)
    a_sum, _ = t_a(a_film)
    m_att = ModalityAttention(feature_dim)
    fused, _ = m_att([v_sum, a_sum])
    x = layers.Dense(512, activation='relu')(fused)
    x = layers.Dropout(0.5)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model([visual_inp, audio_inp], out)

In [ ]:
# ============================================================
# STEP 6️⃣ Load Features and Train Model
# ============================================================
def load_features_and_labels(feature_dir, label_csv, num_classes=28):
    df = pd.read_csv(label_csv)
    Xv, Xa, y = [], [], []
    for _, row in df.iterrows():
        name, label = row['video'], row['label']
        try:
            v = np.load(os.path.join(feature_dir, f"{name}_visual.npy"))
            a = np.load(os.path.join(feature_dir, f"{name}_audio.npy"))
            Xv.append(v)
            Xa.append(a)
            y.append(label)
        except: continue
    Xv, Xa = np.array(Xv), np.array(Xa)
    y = tf.keras.utils.to_categorical(y, num_classes)
    return Xv, Xa, y

# Example training (uncomment below to run):
# Xv, Xa, y = load_features_and_labels(FEATURE_PATH, LABELS_CSV, num_classes=28)
# model = build_mafnet()
# model.compile(optimizer=optimizers.Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
# history = model.fit([Xv, Xa], y, batch_size=8, epochs=50, validation_split=0.1)
# model.save(f"{PROJECT_PATH}/MAFnet_trained_AVE.h5")
# print("✅ Model trained and saved!")

In [ ]:
# ============================================================
# STEP 7️⃣ Visualize Results using t-SNE
# ============================================================
# Visualize the learned multimodal embeddings before and after the FiLM layer to verify cluster separation.
def tsne_visualization(model, Xv, Xa, y_labels):
    extractor = models.Model(inputs=model.inputs, outputs=model.get_layer(index=-3).output)
    features = extractor.predict([Xv, Xa], verbose=0)
    tsne = TSNE(n_components=2).fit_transform(features)
    plt.figure(figsize=(8,6))
    scatter = plt.scatter(tsne[:,0], tsne[:,1], c=np.argmax(y_labels, axis=1), cmap='tab20', s=10)
    plt.title("t-SNE Visualization of Fused Audio-Visual Features")
    plt.colorbar(scatter)
    plt.show()

print("✅ MAFnet full Colab pipeline ready to run! Follow the steps above to train and visualize results.")
